In [95]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
import os
from settings.key import GPT_API_KEY
os.environ["OPENAI_API_KEY"] = GPT_API_KEY

In [117]:
llm = OpenAI(
    model_name="gpt-3.5-turbo",
   #model_name="gpt-4-0613",
                streaming=True,
                verbose=True,
                callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), 
             )
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/langchain/llms/openai.py:158: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/langchain/llms/openai.py:661: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [118]:

#resp = llm("絶望する歌詞を書いてください。")

In [119]:
#conversation("明日の天気は?")

In [120]:
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re


In [121]:

from LLMSearch.BM25VecDB import BM25VecDB
bmdb=BM25VecDB()

In [122]:
import random
def lit_search(query:str ) -> str:
    ans_list=bmdb.search(query,k=10)

    short_list=[]
    for ans in ans_list:
        ref_path=ans["path"].split("/")[-1]
        ref_path=re.sub(r"\.txt", "", ref_path)
        short_list.append((ref_path+":"+ans["text"]))

    final_list=[short_list[0]]

    #残り3件をランダムに選ぶ
    final_list=final_list+random.sample(short_list[1:],3)
    return str(final_list)


custom_tools = [
    Tool(
        name=f"search",
        func=lit_search,
        description=f"search",
    )
]
ALL_TOOLS = custom_tools 

In [123]:
#embed retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(ALL_TOOLS)
]

vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

In [124]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

output_parser = CustomOutputParser()

In [125]:
from typing import Callable

# Set up the base template
template = """Answer following questions as best you can.
You only have access to the following tools:
{tools}

Use the following format:

Question: input question you must answer
Thought: you should always think about what to do
Action: action to take, should be one of [{tool_names}]. Only JAPANESE is supported.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Speak in Japanese

Question: {input}
{agent_scratchpad}"""

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [126]:
tools = get_tools("search")
llm_chain = LLMChain(llm=llm, prompt=prompt)


tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [127]:

agent_executor.run("フリーラジカル重合の課題は?")





> Entering new AgentExecutor chain...
Thought: フリーラジカル重合について情報を探す必要がある。
Action: search
Action Input: "フリーラジカル重合　課題"Thought: フリーラジカル重合について情報を探す必要がある。
Action: search
Action Input: "フリーラジカル重合　課題"training model...


Observation:["book_高分子の合成（上）:631)3章フリーラジカル重合の素反応(cid.631)･…･(cid.631)(cid.631)…･･･““……(cid.631)･(cid.631)･(cid.631)…3.1開始反応・･･･…･…･…･“(cid.631)･･･(cid.631)･3.1.1開始剤・‘･･･(cid.631)･･･(cid.631).,.････…･･(cid.631)･･･････…･･･…･････(cid.631)･･･(cid.631)･････････…･…･････(cid.631)･･･'･(cid.631)･･(cid.631)･.3.1.2開始反応速度と末端基の検出(cid.631)…(cid.631)(cid.631)…………(cid.631)…(cid.631)…3.2生長反応･･･…･……･…･････…..…･････……････････…..3.2.1生長以応速度定数の決定…･･………………………3(cid.631)2(cid.631)2種々のモノマーの生長反応速度定数…･(cid.631)･･･(cid.631)…(cid.", 'book_分子から材料までどんどんつながる高分子:E.Hudgin,ed.,MarcelDekker(1996)4.2ラジカル重合ラジカル重合は利用可能なモノマーの種類がアニオン重合（儲.p.101)やカチオン重合（認-"p.106)と比べて多いため，ビニル化合三ノ.944章尚分子の合成物の重合ではもっとも利用されている.しかし高活性なラジカルを利用した重合法であるため精密な重合制御は困難とされてきた.現在ではリビング重合（絲嵯p.116)のように重合が制御できるラジカル重合法が確立されているが，本節ではもっとも一般的に用いられているラジカル重

INFO:openai:error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 4362 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4362 tokens. Please reduce the length of the messages.